In [ ]:
path = './nlp_test_task_2022/dataset/'

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') 
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def clean_data(text):
    p = re.compile(r'[^А-ЯËа-яёA-Za-z0-9]')
    return re.sub(p, ' ', str(text)).lower()

### Обучающая выборка

In [ ]:
data = pd.read_csv(f'{path}train.tsv', sep='\t')
data

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1
...,...,...
5753,Эдди Чемберс получил сотрясение мозга в бою с ...,0
5754,Правительство застроит Россию нефтепродуктопро...,0
5755,«Мне стыдно перед дедом»: новый канцлер ФРГ об...,1
5756,Туркмения в декабре начнет поставки газа в Китай,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5758 entries, 0 to 5757
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    5758 non-null   object
 1   is_fake  5758 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 90.1+ KB


In [ ]:
X, y = data['title'], data['is_fake']

In [ ]:
X.describe()

count                                                  5758
unique                                                 5757
top       В США зафиксировано рекордное количество банкр...
freq                                                      2
Name: title, dtype: object

In [ ]:
set(X.value_counts())

{1, 2}

In [ ]:
X.value_counts().value_counts()

1    5756
2       1
Name: title, dtype: int64

In [ ]:
X.describe()['top']

'В США зафиксировано рекордное количество банкротств'

In [ ]:
data[data['title'] == X.describe()['top']]

,title,is_fake
402,В США зафиксировано рекордное количество банкр...,0
624,В США зафиксировано рекордное количество банкр...,0


In [ ]:
dt = data.copy()

In [ ]:
dt['len_title'] = list(map(lambda x: len(x), dt['title'].values))

In [ ]:
dt.head(3)

,title,is_fake,len_title
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1,77
1,Агент Кокорина назвал езду по встречке житейск...,0,57
2,Госдума рассмотрит возможность введения секрет...,1,75


In [ ]:
dt.groupby('is_fake')['len_title'].agg(np.mean)

is_fake
0    53.798541
1    76.454672
Name: len_title, dtype: float64

In [ ]:
dt.groupby('is_fake')['len_title'].agg(np.median)

is_fake
0    53.0
1    75.0
Name: len_title, dtype: float64

В среднем заголовки фейковых новостей длиннее. 

In [ ]:
not_fake = dt[dt['is_fake'] == 0]['title'].agg(clean_data)
fake = dt[dt['is_fake'] == 1]['title'].agg(clean_data)

In [ ]:
stop_en = stopwords.words('english')
stop_ru = stopwords.words('russian')

In [ ]:
nf_words, f_words = [], []
for sent in not_fake.apply(lambda string: nltk.word_tokenize(string)):
    sent = [word for word in sent if word not in stop_en and word not in stop_ru]
    nf_words.extend(sent)

for sent in fake.apply(lambda string: nltk.word_tokenize(string)):
    sent = [word for word in sent if word not in stop_en and word not in stop_ru]
    f_words.extend(sent)

In [ ]:
count_nf_words = Counter(nf_words)
count_f_words = Counter(f_words)

In [ ]:
popular_nf = sorted(count_nf_words, key=count_nf_words.get, reverse = True)
popular_f = sorted(count_f_words, key=count_f_words.get, reverse = True)

In [ ]:
popular_nf[:20]  # самые популярные слова в реальных заголовках

['россии',
 'сша',
 'сборной',
 'долларов',
 'мира',
 'года',
 'россия',
 'рублей',
 'москве',
 'евро',
 'российский',
 'путин',
 'новый',
 'ученые',
 'чемпионата',
 'миллионов',
 'процентов',
 'российские',
 'матче',
 'стал']

In [ ]:
count_nf_words['россии'], count_nf_words['стал']

(164, 25)

In [ ]:
popular_nf[-20:]  # самые непопулярные слова в реальных заголовках

['резервах',
 'горбачев',
 'встретился',
 'гельмутом',
 'колем',
 'съемок',
 'телесериала',
 'сознание',
 'retribution',
 'избежала',
 'рецессии',
 'профессиональный',
 'чемберс',
 'сотрясение',
 'мозга',
 'бою',
 'застроит',
 'нефтепродуктопроводами',
 'туркмения',
 'нальчикский']

In [ ]:
popular_f[:20]  # самые популярные слова в фейковых заголовках

['россии',
 'навального',
 'россиян',
 'сша',
 'суд',
 'рублей',
 'года',
 'будут',
 'запретят',
 'предложили',
 'против',
 'лукашенко',
 'москве',
 'госдуме',
 '1',
 'предложил',
 'россия',
 'российских',
 'москвы',
 'рпц']

In [ ]:
count_f_words['россии'], count_f_words['навального']

(237, 82)

In [ ]:
popular_f[-20:]  # самые непопулярные слова в фейковых заголовках

['имуществом',
 'измену',
 'косметикой',
 'космонавт',
 'акция',
 'угрожает',
 'стабильной',
 'опубликовала',
 'доносов',
 'якубович',
 'версии',
 'крупной',
 'растворился',
 'тумане',
 'устроивший',
 'пробку',
 'стыдно',
 'дедом',
 'речью',
 'бундестагу']

In [ ]:
y.value_counts()

1    2879
0    2879
Name: is_fake, dtype: int64

In [ ]:
d = data.copy()

In [ ]:
titles = []
for line in d['title']:
    titles.extend(list(line.lower()))

In [ ]:
len(set(titles))

98

In [ ]:
# set(titles)

In [ ]:
p = re.compile(r'[^А-ЯËа-яёA-Za-z0-9(),!?@\_\n]')
re.findall(p, ''.join(set(titles)))

['»',
 '-',
 '–',
 '°',
 '+',
 '#',
 '«',
 '$',
 '—',
 'é',
 '’',
 '.',
 '*',
 '\u200e',
 '`',
 ':',
 '/',
 '%',
 ' ',
 '”',
 ';',
 '“',
 '&']

In [ ]:
def clean_data(text):
    p = re.compile(r'[^А-ЯËа-яёA-Za-z0-9(),!?@\_\n]')
    return re.sub(p, ' ', str(text)).lower()
 
d['title_new'] = d['title'].agg(clean_data)

In [ ]:
d

,title,is_fake,title_new
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1,москвичу владимиру клутину пришёл счёт за вмеш...
1,Агент Кокорина назвал езду по встречке житейск...,0,агент кокорина назвал езду по встречке житейск...
2,Госдума рассмотрит возможность введения секрет...,1,госдума рассмотрит возможность введения секрет...
3,ФАС заблокировала поставку скоростных трамваев...,0,фас заблокировала поставку скоростных трамваев...
4,Против Навального завели дело о недоносительст...,1,против навального завели дело о недоносительст...
...,...,...,...
5753,Эдди Чемберс получил сотрясение мозга в бою с ...,0,эдди чемберс получил сотрясение мозга в бою с ...
5754,Правительство застроит Россию нефтепродуктопро...,0,правительство застроит россию нефтепродуктопро...
5755,«Мне стыдно перед дедом»: новый канцлер ФРГ об...,1,мне стыдно перед дедом новый канцлер фрг об...
5756,Туркмения в декабре начнет поставки газа в Китай,0,туркмения в декабре начнет поставки газа в китай


In [ ]:
d.iloc[219]

title        «Когда нас на автобусах возили к президенту, в...
is_fake                                                      1
title_new     когда нас на автобусах возили к президенту, в...
Name: 219, dtype: object

In [ ]:
d.drop(index=402, inplace=True)

In [ ]:
d.iloc[:, 1:].to_csv(f'{path}clean_data.csv', index=False)

Я считаю, что удалять стоп-слова из заголовков не стоит, так как информации в предложении и без того не очень много. Возможно, предлоги, союзы и другие стоп-слова будут важны для модели конкретно в этой задаче. (В векторайзерах я все же попробовала убирать стоп-слова, качество моделей становилось хуже). В clean_data я не стала применять лемматизацию, но при обучении в черновике все же добавляла, качество стало чуть хуже, так что отказалась.

### Тестовая выборка (для сдачи задания)

In [ ]:
data = pd.read_csv(f'{path}test.tsv', sep='\t')
data

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,0
1,Ночью под Минском на президентской горе Белара...,0
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,0
3,"Сельская церковь, собравшая рекордно низкое ко...",0
4,Акции Google рухнули после объявления о переза...,0
...,...,...
995,Прокуратура заподозрила Явлинского в авторитар...,0
996,В День Победы стратегические ракетоносцы Ту-16...,0
997,СК возбудил дело против авиакомпании «Победа» ...,0
998,Криптомонетный двор Туркменистана выпустил юби...,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1000 non-null   object
 1   is_fake  1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [ ]:
X, y = data['title'], data['is_fake']

In [ ]:
X.describe()

count                                                  1000
unique                                                 1000
top       Роскомнадзор представил реестр сочетаний цвето...
freq                                                      1
Name: title, dtype: object

In [ ]:
d = data.copy()

In [ ]:
titles = []
for line in d['title']:
    titles.extend(list(line.lower()))

In [ ]:
len(set(titles))

90

In [ ]:
p = re.compile(r'[^А-ЯËа-яёA-Za-z0-9(),!?@\_\n]')
re.findall(p, ''.join(set(titles)))

['»',
 '-',
 '–',
 '#',
 '«',
 '—',
 '.',
 '*',
 ':',
 '/',
 'ë',
 '%',
 ' ',
 '”',
 '“',
 '&']

In [ ]:
d['title_new'] = d['title'].agg(clean_data)

In [ ]:
d

,title,is_fake,title_new
0,Роскомнадзор представил реестр сочетаний цвето...,0,роскомнадзор представил реестр сочетаний цвето...
1,Ночью под Минском на президентской горе Белара...,0,ночью под минском на президентской горе белара...
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,0,бывший спичрайтер юрия лозы рассказал о трудно...
3,"Сельская церковь, собравшая рекордно низкое ко...",0,"сельская церковь, собравшая рекордно низкое ко..."
4,Акции Google рухнули после объявления о переза...,0,акции google рухнули после объявления о переза...
...,...,...,...
995,Прокуратура заподозрила Явлинского в авторитар...,0,прокуратура заподозрила явлинского в авторитар...
996,В День Победы стратегические ракетоносцы Ту-16...,0,в день победы стратегические ракетоносцы ту 16...
997,СК возбудил дело против авиакомпании «Победа» ...,0,ск возбудил дело против авиакомпании победа ...
998,Криптомонетный двор Туркменистана выпустил юби...,0,криптомонетный двор туркменистана выпустил юби...


In [ ]:
d.iloc[:, 1:].to_csv(f'{path}clean_data_test.csv', index=False)